In [9]:
#Brker CNN arkitektur kombinert med PyTorch 


In [10]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split

In [11]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n5_M8_res50_10000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_10000_events.h5")

In [12]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [13]:
dataArray = np.concatenate((bhArray,sphArray),axis=0)

In [14]:
np.shape(dataArray)

(20000, 50, 50, 3)

In [15]:
labelsArray = np.concatenate((np.zeros(10_000),np.ones(10_000)),axis=0)

In [16]:
np.shape(labelsArray)

(20000,)

In [17]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [18]:
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [19]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [20]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [21]:
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

In [47]:
class CNNModel(nn.Module):
    def __init__(self, resolution, num_classes, stride=1):

        super(CNNModel, self).__init__()
        self.fc1 = nn.Linear(3, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 2)
        


    def forward(self, x:Tensor):
        out = F.relu(self.fc1(x))
        out = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out

In [48]:
model = CNNModel(50*50*3, 2)

In [49]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [50]:
num_epochs = 3
for epoch in range(num_epochs):
    y_pred = model(trainData)
    loss = criterion(y_pred)

    loss.backwards()

    optimizer.step()

    optimizer.zero_grad()

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 150000000000 bytes.